# 증권사 API 연결과 매매하기

## 접근 토큰 및 해시키 발급하기

In [1]:
import requests
import json
import keyring

# key
app_key = keyring.get_password('koreainvest_mock_app_key', 'ahn283')
app_secret = keyring.get_password('koreainvest_mock_app_secret', 'ahn283')

# base url
url_base = 'https://openapivts.koreainvestment.com:29443'        # 모의투자

# information
headers = {"content-type": "application/json"}
path = "oauth2/tokenP"
body = {
    "grant_type": "client_credentials",
    "appkey": app_key,
    "appsecret": app_secret
}
url = f"{url_base}/{path}"
print(url)

https://openapivts.koreainvestment.com:29443/oauth2/tokenP


In [2]:
# 위 url을 body에 전송하면 접근 토근을 받을 수 있다.
# dumps() 함수를 통해 딕셔너리를 json 문자열로 변환하여 전송
res = requests.post(url, headers=headers, data=json.dumps(body))
access_token = res.json()['access_token']

### 해시키 발급받기

In [3]:
def hashkey(datas):
    path = 'uapi/hashkey'
    url = f'{url_base}/{path}'
    headers = {
        'content-type': 'application/json',
        'appKey': app_key,
        'appSecret': app_secret
    }
    res = requests.post(url, headers=headers, data=json.dumps(datas))
    hashkey = res.json()['HASH']
    
    return hashkey

## 주식 현재가 시세 조회하기

In [4]:
# 삼성전자

path = 'uapi/domestic-stock/v1/quotations/inquire-price'
url = f'{url_base}/{path}'

headers = {
    'Content-type' : 'application/json',
    'authorization' : f'Bearer {access_token}',
    'appKey' : app_key,
    'appSecret' : app_secret,
    # tr_ud는 거래ID에 해당하는 부분으로써 주식현재가 시세에 해당하는 FHKST0101010을 입력한다.
    'tr_id' : 'FHKST01010100'
}
# fid_cond_mrkt_div_code는 시장 분류코드며, 주식에 해당하는 J를 입력하고 fid_input_iscd에는 삼성전자 티커를 입력
params = {'fid_cond_mrkt_div_code': 'J', 'fid_input_iscd': '005930'}

res = requests.get(url, headers=headers, params=params)
res.json()['output']['stck_prpr']

'72600'

## 주식 주문하기

### 매수주문

In [5]:
path = '/uapi/domestic-stock/v1/trading/order-cash'
url = f'{url_base}/{path}'

data = {
    'CANO': '50099655',     # 계좌번호 앞 8자리
    'ACNT_PRDT_CD': '01',   # 계좌번호 뒤 2자리
    'PDNO': '005930',       # 종목코드
    'ORD_DVSN': "01",       # 주문 방법 : 01-시장가 주문
    'ORD_QTY': "10",        # 주문   
    'ORD_UNPR': "0"         # 주문 단가 (시장가의 경우 0)    
}

headers = {
    "Content-Type": "application/json",
    "authorization": f"Bearer {access_token}",
    "appKey": app_key,
    "appSecret": app_secret,
    "tr_id": "VTTC0802U",               # 주식 현금 매수 주문 tr_id
    "custtype": "P",
    "hashkey": hashkey(data)            # 주문 보안을 위해 해시키를 발급받아 입력
}

# post() 함수를 통해 요청
res = requests.post(url, headers=headers, data=json.dumps(data))
res.json()

{'rt_cd': '1', 'msg_cd': '40580000', 'msg1': '모의투자 장종료 입니다.'}

### 정정 주문

In [6]:
path = '/uapi/domestic-stock/v1/trading/order-cash'
url = f'{url_base}/{path}'

data = {
    'CANO': '50099655',     # 계좌번호 앞 8자리
    'ACNT_PRDT_CD': '01',   # 계좌번호 뒤 2자리
    'PDNO': '005930',       # 종목코드
    'ORD_DVSN': "00",       # 주문 방법
    'ORD_QTY': "10",        # 주문 수량
    'ORD_UNPR': "50000",    # 주문 단가 (시장가의 경우 0)
}

headers = {
    'Content-Tye': 'application/json',
    'authorization': f'Bearer {access_token}',
    'appKey': app_key,
    'appSecret': app_secret,
    'tr_id': 'VTTC0802U',
    'custtype': 'P',
    'hashkey': hashkey(data)
}

res = requests.post(url, headers=headers, data=json.dumps(data))
res.json()

{'rt_cd': '1', 'msg_cd': '40580000', 'msg1': '모의투자 장종료 입니다.'}

In [8]:
# 한국거래소전송주문 조직번호과 주문번호
# res에서 구할 수 있음
KRX_FWDG_ORGNO = res.json()['output']['KRX_FWDG_ORD_ORGNO']     # 한국거래소전송주문조직번호
ODNO = res.json()['output']['ODNO']         # 주문번호

In [ ]:
# 미체결된 주문을 최유리 지정가로 변경 정정 주문
path = 'uapi/domestic-stock/v1/trading/order-rvsecncl'
url = f'{url_base}/{path}'

data = {
    'CANO': '50099655',     # 계좌번호 앞 8자리
    'ACNT_PRDT_CD': '01',   # 계좌번호 뒤 2자리
    'KRX_FWDG_ORD_ORGNO': KRX_FWDG_ORGNO,   # 한국거래소전송주문 조직번호
    'ORGN_ODNO': ODNO,      # 주문번호
    'ORD_DVSN': '03',       # 주문 방법
    'RVSE_CNCL_DVSN_CD': '01',  # 정정 (취소는 02)
    'ORD_QTY': '10',        # 주문 수량
    'ORD_UNPR': '0',        # 주문 단가 (시장가의 경우 0)
    'QTY_ALL_ORD_YN': 'Y'   # 잔량 전부 (잔량 일부는 N)    
}

headers = {
    'Content-Type': 'application/json',
    'authorization': f'Bearer {access_token}',
    'appKey': app_key,
    'appSecret': app_secret,
    'tr_id': 'VTTC0803U',
    'custtype': 'P',
    'hashkey': hashkey(data)
}

res = requests.post(url, headers=headers, data=json.dump(data))
res.json()

### 매도 주문

In [ ]:
# 삼성전자 10주 시장가 매도
path = '/uapi/domestic-stock/v1/trading/order-cash'
url = f'{url_base}/{path}'

data = {
    'CANO': '50099655',     # 계좌번호 앞 8자리
    'ACNT_PRDT_CD': '01',   # 계좌번호 뒤 2자리
    'PDNO': '005930',       # 종목코드
    'ORD_DVSN': '01',       # 주문 방법
    'ORD_QTY': '10',        # 주문 수량
    'ORD_UNPR': '0',        # 주문 단가 (시장가의 경우 0)    
}

headers = {
    'Content-Type': 'application/json',
    'authorization': f'Bearer {access_token}',
    'appKey': app_key,
    'appSecret': app_secret,
    'tr_id': 'VTTC0801U',
    'custtype': 'P',
    'hashkey': hashkey(data)
}

res = requests.post(url, headers=headers, data=json.dumps(data))

## 주식 잔고조회

In [10]:
path = '/uapi/domestic-stock/v1/trading/inquire-balance'
url = f'{url_base}/{path}'

headers = {
    'Content-Typoe': 'application/json',
    'authorization': f'Bearer {access_token}',
    'appKey': app_key,
    'appSecret': app_secret,
    'tr_id': 'VTTC8434R'
}

params = {
    'CANO': '50099655',     # 계좌번호 앞 8자리
    'ACNT_PRDT_CD': '01',   # 계좌번호 뒤 2자리
    'AFHR_FLPR_YN': 'N',    # 시간외단일가여부
    'OFL_YN': '',           # 공란
    'INQR_DVSN': '01',      # 조회구분
    'UNPR_DVSN': '01',      # 단가구분
    'FUND_STTL_ICLD_YN': 'N',   # 펀드결제분포함여부
    'FNCG_AMT_AUTO_RDPT_YN': 'N',    # 융자금액자동상환여부
    'PRCS_DVSN': '00',      # 처리구분 (00: 전일매매포함)
    'CTX_AREA_FK100': '',   # 연속조회검색조건
    'CTX_AREA_NK100': '',   # 연속조회키    
}

# get 함수 통해 데이터를 요청
res = requests.get(url, headers=headers, params=params)

In [13]:
# ['output1']에는 보유 종목에 대한 정보
res.json()['output1']

[]

In [14]:
# ['output2']에는 계좌정보가 들어 있다.
res.json()['output2']

[{'dnca_tot_amt': '500000000',
  'nxdy_excc_amt': '500000000',
  'prvs_rcdl_excc_amt': '500000000',
  'cma_evlu_amt': '0',
  'bfdy_buy_amt': '0',
  'thdt_buy_amt': '0',
  'nxdy_auto_rdpt_amt': '0',
  'bfdy_sll_amt': '0',
  'thdt_sll_amt': '0',
  'd2_auto_rdpt_amt': '0',
  'bfdy_tlex_amt': '0',
  'thdt_tlex_amt': '0',
  'tot_loan_amt': '0',
  'scts_evlu_amt': '0',
  'tot_evlu_amt': '500000000',
  'nass_amt': '500000000',
  'fncg_gld_auto_rdpt_yn': '',
  'pchs_amt_smtl_amt': '0',
  'evlu_amt_smtl_amt': '0',
  'evlu_pfls_smtl_amt': '0',
  'tot_stln_slng_chgs': '0',
  'bfdy_tot_asst_evlu_amt': '500000000',
  'asst_icdc_amt': '0',
  'asst_icdc_erng_rt': '0.00000000'}]

In [15]:
# 티커 종목명, 보유수량, 매입평균가격을 데이터 프레임으로 변환
import pandas as pd

ap = pd.DataFrame.from_records(res.json()['output1'])
ap

""


## 스케쥴링

In [2]:
import datetime

def job():
    print(datetime.datetime.now().strftime('%H:%M:%S'))
    print("=================")

In [3]:
# 매 3초마다 job() 함수 실행

import schedule
schedule.every(3).seconds.do(job)

Every 3 seconds do job() (last run: [never], next run: 2023-12-08 23:28:21)

In [4]:
# 등록된 스케쥴 확인
schedule.get_jobs()

[Every 3 seconds do job() (last run: [never], next run: 2023-12-08 23:28:21)]

In [20]:
# # 함수 실행
# while True:
#     schedule.run_pending()

18:35:23
18:35:26
18:35:29
18:35:32
18:35:35


: 

### 시간 지정하기

In [5]:
# 작업이 실행될 시간과 종료할 시간을 직접 지정

import pandas as pd
from datetime import timedelta

schedule.clear()

startDt = datetime.datetime.now() + timedelta(seconds=60)
endDt = datetime.datetime.now() + timedelta(seconds=80)
# date_range 함수를 통해 startDt와 endDt를 10개 구간으로 나눈다.
time_list = pd.date_range(startDt, endDt, periods=5)

print(time_list)

DatetimeIndex(['2023-12-08 23:29:25.967518', '2023-12-08 23:29:30.967518',
               '2023-12-08 23:29:35.967518', '2023-12-08 23:29:40.967518',
               '2023-12-08 23:29:45.967518'],
              dtype='datetime64[ns]', freq=None)


In [8]:
# 스케쥴러 등록에 필요한 %H:%M:%S 형태로 변환
time_list_sec = [i.strftime('%H:%M:%S') for i in time_list]
time_list_sec

['23:29:25', '23:29:30', '23:29:35', '23:29:40', '23:29:45']

In [9]:
# job() 함수를 수행하도록 스케쥴을 등록
[schedule.every().day.at(i).do(job) for i in time_list_sec]

[Every 1 day at 23:29:25 do job() (last run: [never], next run: 2023-12-08 23:29:25),
 Every 1 day at 23:29:30 do job() (last run: [never], next run: 2023-12-08 23:29:30),
 Every 1 day at 23:29:35 do job() (last run: [never], next run: 2023-12-08 23:29:35),
 Every 1 day at 23:29:40 do job() (last run: [never], next run: 2023-12-08 23:29:40),
 Every 1 day at 23:29:45 do job() (last run: [never], next run: 2023-12-08 23:29:45)]

In [10]:
while True:
    schedule.run_pending()
    if datetime.datetime.now() > endDt:
        print('End')
        schedule.clear()
        break

23:29:25
23:29:30
23:29:35
23:29:40
23:29:45
End


## 포트폴리오 리밸런싱

### 포트폴리오 매수

In [1]:
# 국내 대형주 10개로 구성된 포트폴리오 시분할 매수
import requests
import json
import keyring
import pandas as pd
import time
import numpy as np
import datetime
from datetime import timedelta
import schedule

# API key
app_key = keyring.get_password('koreainvest_mock_app_key', 'ahn283')
app_secret = keyring.get_password('koreainvest_mock_app_secret', 'ahn283')

# access token
url_base = 'https://openapivts.koreainvestment.com:29443'

headers = {'content-type': 'application/json'}
path = 'oauth2/tokenP'
body = {
    'grant_type': 'client_credentials',
    'appkey': app_key,
    'appsecret': app_secret
}

url = f'{url_base}/{path}'
res = requests.post(url, headers=headers, data=json.dumps(body))
access_token = res.json()['access_token']

# hash key
def hashkey(datas):
    path = 'uapi/hashkey'
    url = f'{url_base}/{path}'
    headers = {
        'content-type': 'application/json',
        'appKey': app_key,
        'appSecret': app_secret
    }
    res = requests.post(url, headers=headers, data=json.dumps(data))
    hashkey = res.json()['HASH']
    
    return hashkey
    

In [2]:
# 현재가 조회 함수
def get_price(ticker):
    path = 'uapi/domestic-stock/v1/quotations/inquire-price'
    url = f'{url_base}/{path}'
    
    headers = {
        'content-type': 'application/json',
        'authorization': f'Bearer {access_token}',
        'appKey': app_key,
        'appSecret': app_secret,
        'tr_id': 'FHKST01010100'
    }
    
    params = {
        'fid_cond_mrkt_div_code': 'J',
        'fid_input_iscd': ticker
    }
    
    res = requests.get(url, headers=headers, params=params)
    price = res.json()['output']['stck_prpr']
    price = int(price)
    # API는 초당 10건까지 요청할 수 있으므로, 연속조회를 위해 0.1초의 정지를 준다.
    time.sleep(0.1)
    
    return price

In [3]:
# 주문하기 함수
def trading(ticker, tr_id):
    
    path = '/uapi/domestic-stock/v1/trading/order-cash'
    url = f'{url_base}/{path}'
    
    data = {
        'CANO': '50099655',     # 계좌번호 앞 8자리
        'ACNT_PRDT_CD': '01',   # 계좌번호 뒤 2자리  
        'PDNO': ticker,
        'ORD_DVSN': '03',
        'ORD_QTY': '1',
        'ORD_UNPR': '0'      
    }
    
    headers = {
        'Content-Type': 'application/json',
        'authorization': f'Bearer {access_token}',
        'appKey': app_key,
        'appSecret': app_secret,
        'tr_id': tr_id,
        'custtype': 'P',
        'hashkey': hashkey(data)
    }
    
    res = requests.post(url, headers=headers, data=json.dumps(data))

In [4]:
# 계좌 잔고 조회 함수

def check_account():
    
    output1 = []
    output2 = []
    CTX_AREA_NK100 = ''
    
    # 모의투자에서는 20종목까지, 실제계좌는 50종목까지 조회가 가능
    # 연속조회 값이 없을 경우 res.json()['ctx_area_nk100'] 값이 ''이므로 ''이 나올떄까지 while문 반복
    while True:
        path = '/uapi/domestic-stock/v1/trading/inquire-balance'
        url = f'{url_base}/{path}'
        
        headers = {
            'Content-Type': 'application/json',
            'authorization': f'Bearer {access_token}',
            'appKey': app_key,
            'appSecret': app_secret,
            'tr_id': 'VTTC8434R'
        }
        
        params = {
            'CANO': '50099655',     # 계좌번호 앞 8자리
            'ACNT_PRDT_CD': '01',   # 계좌번호 뒤 2자리 
            'AFHR_FLPR_YN': 'N',
            'UNPR_DVSN': '01',
            'FUND_STTL_ICLD_YN': 'N',
            'FNCG_AMT_AUTO_RDPT_YN': 'N',
            'OFL_YN': "",
            'INQR_DVSN': '01',
            'PRCS_DVSN': '00',
            'CTX_AREA_FK100': '',
            'CTX_AREA_NK100': CTX_AREA_NK100        
        }
        
        res = requests.get(url, headers=headers, params=params)
        output1.append(pd.DataFrame.from_records(res.json()['output1']))
        CTX_AREA_NK100 = res.json()['ctx_area_nk100'].strip()
        
        if CTX_AREA_NK100 == '':
            output2.append(res.json()['output2'][0])
            break
    
    # 보유종목이 없는 경우에는 빈데이터프레임 입력
    if not output1[0].empty:
        res1 = pd.concat(output1)[['pdno', 'hldg_qty']].rename(columns={
            'pdno': '종목코드', 'hldg_qty': '보유수량'                
        }).reset_index(drop=True)
    else:
        res1 = pd.DataFrame(columns=['종목코드', '보유수량'])
    
    res2 = output2[0]
    
    return [res1, res2]

In [5]:
ap, account = check_account()
print(ap, account)

Empty DataFrame
Columns: [종목코드, 보유수량]
Index: [] {'dnca_tot_amt': '500000000', 'nxdy_excc_amt': '500000000', 'prvs_rcdl_excc_amt': '500000000', 'cma_evlu_amt': '0', 'bfdy_buy_amt': '0', 'thdt_buy_amt': '0', 'nxdy_auto_rdpt_amt': '0', 'bfdy_sll_amt': '0', 'thdt_sll_amt': '0', 'd2_auto_rdpt_amt': '0', 'bfdy_tlex_amt': '0', 'thdt_tlex_amt': '0', 'tot_loan_amt': '0', 'scts_evlu_amt': '0', 'tot_evlu_amt': '500000000', 'nass_amt': '500000000', 'fncg_gld_auto_rdpt_yn': '', 'pchs_amt_smtl_amt': '0', 'evlu_amt_smtl_amt': '0', 'evlu_pfls_smtl_amt': '0', 'tot_stln_slng_chgs': '0', 'bfdy_tot_asst_evlu_amt': '500000000', 'asst_icdc_amt': '0', 'asst_icdc_erng_rt': '0.00000000'}


In [6]:
# 모델 포트폴리오
mp = pd.DataFrame({
    '종목코드': [
        '005930',       # 삼성전자
        '373220',       # LG에너지솔루션
        '000660',       # SK하이닉스
        '207940',       # 삼성바이로직스
        '051910',       # LG화학
        '035420',       # NAVER
        '005380',       # 현대차 
        '006400',       # 삼성SDI
        '035720',       # 카카오 
        '105560',       # KB금융
    ]
})

# 보유 종목과 aum 불러오기
ap, account = check_account()

# 주당 투자 금액
## 총 평가금액의 100% 투자할 경우, 리밸런싱 과정에서 주가의 등락에 의해 주문금액이 보유금액을 초과하는 일이 벌어질 수 있으므로
## 1~5% 정도의 현금은 보유하는 것이 좋다.
invest_per_stock = int(account['tot_evlu_amt']) * 0.98 / len(mp)

# 매매 구성
target = mp.merge(ap, on='종목코드', how='outer')
## 보유하고 있지 않은 종목은 NA로 표시되므로, 이를 0으로 바꾼다.
target['보유수량'] = target['보유수량'].fillna(0).apply(pd.to_numeric)

# 현재가 확인
target['현재가'] = target.apply(lambda x: get_price(x.종목코드), axis=1)

# 현재가 확인
## 모델 포트폴리오에 있는 경우는 '종목당 투자 금액/현재가'를 통해 종목당 목표 수량이 얼마인지 계산
## 모델 포트폴리오에 존재하지 않아 전략 매도해야 하는 종목은 목표수량에 0을 입력
target['목표수량'] = np.where(target['종목코드'].isin(mp['종목코드'].tolist()),
                          round(invest_per_stock / target['현재가']), 0)
target['투자수량'] = target['목표수량'] - target['보유수량']

In [7]:
target

,종목코드,보유수량,현재가,목표수량,투자수량
0,005930,0,72600,675.0,675.0
1,373220,0,430500,114.0,114.0
2,000660,0,127500,384.0,384.0
3,207940,0,709000,69.0,69.0
4,051910,0,478500,102.0,102.0
5,035420,0,217500,225.0,225.0
6,005380,0,185100,265.0,265.0
7,006400,0,450000,109.0,109.0
8,035720,0,51700,948.0,948.0
9,105560,0,51800,946.0,946.0


In [8]:
# 시분할 주문
# 시간 분할
startDt1 = datetime.datetime.now() + timedelta(minutes=1)
startDt2 = datetime.datetime.now().replace(hour=9, minute=10, second=0, microsecond=0)
startDt = max(startDt1, startDt2)
endDt = datetime.datetime.now().replace(hour=15, minute=0, second=0, microsecond=0)

# 스케쥴 초기화
schedule.clear()

# 스케쥴 등록
for t in range(target.shape[0]):
    n = target.loc[t, '투자수량']
    position = 'VTTC0802U' if n > 0 else 'VTTC0801U'
    ticker = target.loc[t, '종목코드']
    
    # 시작시간부터 종료시간까지 투자수량의 절대값에 해당하는 만큼 기간을 나눈다.
    time_list = pd.date_range(startDt, endDt, periods= abs(n))
    time_list = time_list.round(freq= 's').tolist()
    time_list_sec = [s.strftime('%H:%M:%S') for s in time_list]
    
    for i in time_list_sec:
        schedule.every().day.at(i).do(trading, ticker, position)

In [9]:
schedule.get_jobs()

[Every 1 day at 09:56:44 do trading('005930', 'VTTC0802U') (last run: [never], next run: 2023-12-09 09:56:44),
 Every 1 day at 09:57:11 do trading('005930', 'VTTC0802U') (last run: [never], next run: 2023-12-09 09:57:11),
 Every 1 day at 09:57:38 do trading('005930', 'VTTC0802U') (last run: [never], next run: 2023-12-09 09:57:38),
 Every 1 day at 09:58:05 do trading('005930', 'VTTC0802U') (last run: [never], next run: 2023-12-09 09:58:05),
 Every 1 day at 09:58:32 do trading('005930', 'VTTC0802U') (last run: [never], next run: 2023-12-09 09:58:32),
 Every 1 day at 09:58:59 do trading('005930', 'VTTC0802U') (last run: [never], next run: 2023-12-09 09:58:59),
 Every 1 day at 09:59:26 do trading('005930', 'VTTC0802U') (last run: [never], next run: 2023-12-09 09:59:26),
 Every 1 day at 09:59:53 do trading('005930', 'VTTC0802U') (last run: [never], next run: 2023-12-09 09:59:53),
 Every 1 day at 10:00:20 do trading('005930', 'VTTC0802U') (last run: [never], next run: 2023-12-09 10:00:20),
 

In [11]:
# 스케쥴 실행
while True:
    schedule.run_pending()
    if datetime.datetime.now() > endDt:
        print('거래가 완료되었습니다.')
        schedule.clear()
        break